# 🤗 Úvod do Hugging Face Transformers

**Autor:** Praut s.r.o. - AI Integration & Business Automation

Tento notebook vás provede základy práce s knihovnou Hugging Face Transformers - nejpopulárnější knihovnou pro práci s AI modely.

## Co se naučíte:
- Instalace a konfigurace Hugging Face
- Základní pipeline pro různé úlohy
- Práce s modely a tokenizery
- Praktické příklady automatizace

In [ ]:
# Instalace potřebných knihoven
!pip install -q transformers accelerate torch sentencepiece

In [ ]:
# Import základních knihoven
from transformers import pipeline, AutoModel, AutoTokenizer
import torch

# Kontrola dostupnosti GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Používám zařízení: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Paměť: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 1. Pipeline - Nejjednodušší způsob použití modelů

Pipeline je vysokoúrovňové API, které automaticky:
- Stáhne model a tokenizer
- Předzpracuje vstup
- Provede inferenci
- Zpracuje výstup

In [ ]:
# Příklad 1: Sentiment analýza
sentiment_analyzer = pipeline("sentiment-analysis", device=0 if device=="cuda" else -1)

texty = [
    "Tento produkt je naprosto skvělý, jsem velmi spokojený!",
    "Hrozná kvalita, nikdy více nekoupím.",
    "Je to průměrné, nic extra."
]

vysledky = sentiment_analyzer(texty)
for text, vysledek in zip(texty, vysledky):
    print(f"📝 {text}")
    print(f"   → {vysledek['label']} (skóre: {vysledek['score']:.2%})\n")

In [ ]:
# Příklad 2: Generování textu
generator = pipeline("text-generation", model="gpt2", device=0 if device=="cuda" else -1)

prompt = "Umělá inteligence v podnikání přináší"
vysledek = generator(prompt, max_length=50, num_return_sequences=1, do_sample=True)

print(f"📝 Prompt: {prompt}")
print(f"🤖 Vygenerováno: {vysledek[0]['generated_text']}")

In [ ]:
# Příklad 3: Sumarizace textu
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if device=="cuda" else -1)

dlouhy_text = """
Artificial intelligence (AI) is transforming businesses across all industries. 
Companies are using AI to automate repetitive tasks, analyze large datasets, 
and make better decisions. Machine learning models can predict customer behavior, 
optimize supply chains, and detect fraud. Natural language processing enables 
chatbots and virtual assistants to handle customer inquiries 24/7. Computer vision 
is used in quality control, security systems, and autonomous vehicles. The adoption 
of AI is accelerating, with more organizations investing in AI capabilities to 
stay competitive in the digital economy.
"""

souhrn = summarizer(dlouhy_text, max_length=60, min_length=20, do_sample=False)
print(f"📄 Původní text ({len(dlouhy_text)} znaků)")
print(f"📋 Souhrn: {souhrn[0]['summary_text']}")

## 2. Práce s tokenizery

Tokenizer převádí text na čísla (tokeny), které model zpracuje.

In [ ]:
# Načtení tokenizeru
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

text = "Hello, how are you doing today?"

# Tokenizace
tokens = tokenizer.tokenize(text)
print(f"📝 Text: {text}")
print(f"🔤 Tokeny: {tokens}")

# Převod na ID
ids = tokenizer.encode(text)
print(f"🔢 Token IDs: {ids}")

# Zpět na text
decoded = tokenizer.decode(ids)
print(f"📖 Dekódováno: {decoded}")

In [ ]:
# Pokročilá tokenizace pro model
encoded = tokenizer(
    text,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"  # PyTorch tensory
)

print("📦 Výstup tokenizeru:")
for key, value in encoded.items():
    print(f"   {key}: {value.shape}")

## 3. Přímá práce s modelem

In [ ]:
from transformers import AutoModelForSequenceClassification

# Načtení modelu pro klasifikaci
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Přesun na GPU pokud je dostupná
model = model.to(device)

# Manuální inference
text = "I absolutely love this product!"
inputs = tokenizer(text, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.softmax(outputs.logits, dim=-1)

labels = ["NEGATIVE", "POSITIVE"]
for i, prob in enumerate(predictions[0]):
    print(f"{labels[i]}: {prob:.2%}")

## 4. Praktická automatizace: Hromadné zpracování

In [ ]:
import pandas as pd

# Simulace dat z e-shopu
recenze = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "produkt": ["Notebook", "Myš", "Klávesnice", "Monitor", "Sluchátka"],
    "text": [
        "Great laptop, very fast and reliable!",
        "The mouse stopped working after a week.",
        "Perfect keyboard for programming.",
        "Average quality, nothing special.",
        "Best headphones I've ever owned!"
    ]
})

# Hromadná analýza sentimentu
classifier = pipeline("sentiment-analysis", device=0 if device=="cuda" else -1)
results = classifier(recenze["text"].tolist())

recenze["sentiment"] = [r["label"] for r in results]
recenze["confidence"] = [r["score"] for r in results]

print("📊 Automatická analýza recenzí:")
print(recenze.to_string(index=False))

In [ ]:
# Statistiky
print("\n📈 Statistiky:")
print(f"   Pozitivní recenze: {(recenze['sentiment'] == 'POSITIVE').sum()}")
print(f"   Negativní recenze: {(recenze['sentiment'] == 'NEGATIVE').sum()}")
print(f"   Průměrná jistota: {recenze['confidence'].mean():.2%}")

## 5. Dostupné úlohy v pipeline

| Úloha | Popis |
|-------|-------|
| `text-classification` | Klasifikace textu |
| `token-classification` | NER, POS tagging |
| `question-answering` | Odpovídání na otázky |
| `summarization` | Sumarizace |
| `translation` | Překlad |
| `text-generation` | Generování textu |
| `fill-mask` | Doplňování slov |
| `zero-shot-classification` | Klasifikace bez tréninku |
| `image-classification` | Klasifikace obrázků |
| `object-detection` | Detekce objektů |
| `automatic-speech-recognition` | Převod řeči na text |

---
## 🏁 Shrnutí

V tomto notebooku jsme se naučili:
- ✅ Používat `pipeline` pro rychlé experimenty
- ✅ Pracovat s tokenizery
- ✅ Načítat a používat modely přímo
- ✅ Automatizovat zpracování dat

**Další notebook:** Textová klasifikace a NER